In [2]:
for vehicle in Vehicles.values():
    #if vehicle.id == 0:
        #print(vehicle)
        print(vehicle.get_history_decisions())

[2, 2, 1, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3]
[2, 2, 1, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3]
[2, 2, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3]
[2, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 

In [7]:
for vehicle in Vehicles.values():
    if vehicle.get_orders():
        for order in vehicle.get_orders():
            if order.matched_vehicle_id == 8:
                print(vehicle.id)
                

In [8]:
for order in Total_order.values():
    if order.matched_vehicle_id == 8:
        print(order)

Order(id=13, passengers=2, departure=5, destination=2,virtual_departure=5, matched_vehicle_id=8,battery=287.9966258615341, status=Matched)
Order(id=15, passengers=5, departure=4, destination=2,virtual_departure=4, matched_vehicle_id=8,battery=849.64585838219, status=Matched)


In [3]:
for vehicle in Vehicles.values():
    #if vehicle.id == 0:
        print(vehicle)
        #print(vehicle.get_history_decisions())

Vehicle(id=0, time=100, into_city=4, intercity=2, passenger=6, decision=3, battery=13146.289404829768, matched_order=3)
Vehicle(id=1, time=100, into_city=6, intercity=0, passenger=6, decision=3, battery=16927.32952300423, matched_order=1)
Vehicle(id=2, time=100, into_city=3, intercity=2, passenger=4, decision=3, battery=1711.6264989390825, matched_order=1)
Vehicle(id=3, time=100, into_city=3, intercity=2, passenger=2, decision=3, battery=5379.198192799372, matched_order=1)
Vehicle(id=4, time=100, into_city=2, intercity=3, passenger=7, decision=3, battery=10908.07589746097, matched_order=3)
Vehicle(id=5, time=100, into_city=1, intercity=2, passenger=4, decision=3, battery=4067.5100023358427, matched_order=1)
Vehicle(id=6, time=100, into_city=2, intercity=4, passenger=7, decision=3, battery=4591.004037749382, matched_order=2)
Vehicle(id=7, time=100, into_city=6, intercity=4, passenger=7, decision=3, battery=16934.89234412165, matched_order=1)
Vehicle(id=8, time=100, into_city=4, intercit

In [4]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import SETTING
import RL
import importlib
import tool_func
from update import *

num_vehicle = 20
num_order = 10
num_city = 8
TIME = 100
CAPACITY = 7
row = [10, 1, 3, 10]
Vehicles = {}
speed = 20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))

# 初始化
Vehicles = vehicle_generator(num_vehicle, num_city)
orders_unmatched = {}
G = CityGraph(num_city, 0.3, (20, 100))
name = "navie"
cancel_penalty = 300
order_canceled = 0
Total_order = {}
objval = 0.114514
# 记录所有时间点车辆信息
all_vehicle_records = []

for time in range(TIME):
    group = [[], []]
    
    # 记录每个时刻的车辆信息
    vehicle_records = []
    # 生成订单
    Orders = order_generator(num_order, time, num_city-1, CAPACITY, G)
    for order in Orders.values():
        orders_unmatched[order.id] = order
        Total_order[order.id] = order

    orders_virtual = orders_unmatched
    if time == 0:
        city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)

    for vehicle in Vehicles.values():
        # 将车辆信息存入字典
        vehicle_records.append({
            "id": vehicle.id,
            "time": vehicle.time,
            "into_city": vehicle.into_city,
            "intercity": vehicle.intercity,
            "passenger": vehicle.get_capacity(),
            "decision": vehicle.decision,
            "battery": vehicle.battery,
            "whether_city":vehicle.whether_city,
            "matched_order": vehicle.orders,
            "num_matched": vehicle.get_orders(),
            "orders_unmatched" :len(orders_unmatched),
            "object_value": objval
        })

        # 通过条件分组
        if vehicle.whether_city:
            group[0].append(vehicle.id)
        else:
            group[1].append(vehicle.id)

    # 将当前时刻的车辆记录添加到所有记录中
    all_vehicle_records.append(vehicle_records)

    if len(group[0]) != 0:
        temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
        temp_Lower_Layer.get_decision()
        temp_Lower_Layer.constrain_1()
        temp_Lower_Layer.constrain_2()
        temp_Lower_Layer.constrain_3()
        temp_Lower_Layer.constrain_4()
        temp_Lower_Layer.constrain_5()
        temp_Lower_Layer.model.setParam('OutputFlag', 0)
        total_penalty = cancel_penalty * order_canceled
        temp_Lower_Layer.set_objective(matrix, total_penalty)
        temp_Lower_Layer.model.optimize()

        if temp_Lower_Layer.model.status == GRB.OPTIMAL:
            # 打印最优解
            print(f"{time}次，Optimal solution:")
            i = 0
            with open(f"output_{time}.txt", "w") as file:
                for v in temp_Lower_Layer.model.getVars():
                    if v.x == 1:
                        file.write(f"{v.varName} = {v.x}\n")  # 将结果写入文件
            print("Objective value:", temp_Lower_Layer.model.objVal)
            objval = temp_Lower_Layer.model.objVal
        else:
            temp_Lower_Layer.model.computeIIS()
            temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
            print(f"{time}次，No optimal solution found.")
            self_update(Vehicles, G)
            objval = 0.114514
        
        _, var_order = temp_Lower_Layer.get_decision()
        update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
        vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
        order_canceled = update_order(orders_unmatched, time, speed)
       
    else:
        self_update(Vehicles, G)
        print(f"{time}次，没有车")
        vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
        order_canceled = update_order(orders_unmatched, time, speed)
    print(f"{len(orders_unmatched)}订单未被匹配")
# 将记录保存为 CSV 文件
csv_file = "vehicle_records.csv"
with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    # 写入CSV文件的列名
    fieldnames = ["id", "time", "into_city", "intercity", "passenger", "decision"
                  ,"battery", "whether_city","matched_order","order_id","num_matched"
                  ,"orders_unmatched","object_value"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    
    # 遍历所有时刻的车辆记录
    for vehicle_records in all_vehicle_records:
        writer.writerows(vehicle_records)

print(f"Vehicle information saved to {csv_file}")


Set parameter LicenseID to value 2584673


INFO:gurobipy:Set parameter LicenseID to value 2584673


0次，Optimal solution:
Objective value: 84652.0
0订单未被匹配
1次，Optimal solution:
Objective value: 77998.0
0订单未被匹配
2次，Optimal solution:
Objective value: 73557.0
0订单未被匹配
3次，Optimal solution:
Objective value: 45190.0
4订单未被匹配
4次，没有车
14订单未被匹配
5次，没有车
24订单未被匹配
6次，Optimal solution:
Objective value: 51051.0
28订单未被匹配
7次，没有车
38订单未被匹配
8次，Optimal solution:
Objective value: 43137.0
43订单未被匹配
9次，Optimal solution:
Objective value: 63017.0
46订单未被匹配
10次，Optimal solution:
Objective value: 28529.0
53订单未被匹配
11次，Optimal solution:
Objective value: 83577.0
54订单未被匹配
12次，Optimal solution:
Objective value: 65991.0
55订单未被匹配
13次，没有车
65订单未被匹配
14次，没有车
75订单未被匹配
15次，Optimal solution:
Objective value: 41900.0
79订单未被匹配
16次，没有车
89订单未被匹配
17次，Optimal solution:
Objective value: 60471.0
92订单未被匹配
18次，Optimal solution:
Objective value: 26183.0
99订单未被匹配
19次，Optimal solution:
Objective value: 40561.0
104订单未被匹配
20次，Optimal solution:
Objective value: 44956.0
108订单未被匹配
21次，没有车
118订单未被匹配
22次，没有车
128订单未被匹配
23次，Optimal solution:
Objective va

ValueError: dict contains fields not in fieldnames: 'orders_unmatched', 'num_matched', 'object_value'